Sentiment Analysis usingLogistic Regression
We will be implementing logistic regression for sentiment analysis on tweets. Given a tweet, we will decide if it has a positive sentiment or a negative one. Specifically we will: 

* Learn how to extract features for logistic regression given some text
* Implement logistic regression
* Apply logistic regression on a natural language processing task
* Test using logistic regression
* Perform error analysis

We will be using a data set of tweets. Hopefully we will get more than 99% accuracy.  

## Import functions and data

In [1]:
import numpy as np
import pandas as pd

import nltk
from nltk.corpus import twitter_samples, stopwords
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import confusion_matrix, classification_report

import re
import string
from nltk.tokenize import TweetTokenizer
from nltk.stem import PorterStemmer

In [ ]:
# nltk.downloader.Downloader.download_dir = "Your_path"

In [ ]:
# downloads sample twitter dataset. uncomment the line below if running on a local machine.
# nltk.download('twitter_samples', "C:/Users/pulki/OneDrive/Documents/Jupyter/NLP - Deeplearning.ai/nltk_data")

In [ ]:
#Add the path of downloaded data to nltk
nltk.data.path.append("Your_path/nltk_data")

### Prepare the data
* The `twitter_samples` contains subsets of 5,000 positive tweets, 5,000 negative tweets, and the full set of 10,000 tweets.  
    * If we used all three datasets, we would introduce duplicates of the positive tweets and negative tweets.  
    * We will select just the five thousand positive tweets and five thousand negative tweets.

In [2]:
# select the set of positive and negative tweets
all_positive_tweets = twitter_samples.strings('positive_tweets.json')
all_negative_tweets = twitter_samples.strings('negative_tweets.json')

* Train test split: 20% will be in the test set, and 80% in the training set.


In [3]:
# combine positive and negative labels
tweets = all_positive_tweets+all_negative_tweets
len(tweets)

10000

* Create the numpy array of positive labels and negative labels.

In [4]:
labels = np.append(np.ones(len(all_positive_tweets)), np.zeros(len(all_negative_tweets)))

In [5]:
labels

array([1., 1., 1., ..., 0., 0., 0.])

In [6]:
# split the data into two pieces, one for training and one for testing (validation set) 
x_train = all_positive_tweets[1000:] + all_negative_tweets[:-1000]
y_train = labels[1000:9000]
x_test = all_positive_tweets[:1000] + all_negative_tweets[-1000:]
y_test = np.append(labels[:1000], labels[-1000:])

### Process tweet
Define a function `process_tweet()` to tokenize the tweet into individual words, remove stop words and apply stemming.

In [7]:
stopwords_english = stopwords.words('english')
punctuation = string.punctuation

def preprocess_tweet(tweet):
    tweet = re.sub(r'^RT[\s]+', '', tweet)
    tweet = re.sub(r'https?./\/\.*[\r\n]*', '', tweet)
    tweet = re.sub(r'#*', '', tweet)
    
    tokenizer = TweetTokenizer()
    tokens = tokenizer.tokenize(tweet.lower())
    
    clean_tokens = [x for x in tokens if (x not in stopwords_english) & (x not in punctuation)]
    
    stemmer = PorterStemmer()
    return [stemmer.stem(word) for word in clean_tokens]

In [8]:
def build_freq(tweet_list, labels):
    wl_pair = []
    for tweet, label in zip(tweet_list, labels):
        for word in tweet:
            wl_pair.append([word, label])
    
    freq = pd.DataFrame(wl_pair, columns=['vocab', 'label'])
    
    word_freq = pd.DataFrame(freq.groupby('vocab').sum())
    
    freq.label = 1-freq.label
    word_freq['neg'] = freq.groupby('vocab').sum()
    
    return word_freq.convert_dtypes().rename(columns={'label': 'pos'}).reset_index()#.to_dict(orient='list')

In [10]:
# test the function below
freq_df = build_freq([preprocess_tweet(tweet) for tweet in x_train], y_train)
freq_df.head()

,vocab,pos,neg
0,(-:,2,0
1,(:,0,6
2,):,7,6
3,);,1,0
4,--->,1,0


In [11]:
import ctypes

x = id(freq_df)
y = ctypes.cast(x, ctypes.py_object).value

type(y)

pandas.core.frame.DataFrame

## Part 2: Extracting the features

* Given a list of tweets, extract the features and store them in a matrix. We will extract two features.
    * The first feature is the number of positive words in a tweet.
    * The second feature is the number of negative words in a tweet. 
* Then train the logistic regression classifier on these features.
* Test the classifier on a validation set. 

In [12]:
def extract_features(tokenized_tweet):
    '''
    Input: 
        tweet: a list of words for one tweet
        freqs: a dictionary corresponding to the frequencies of each tuple (word, label)
    Output: 
        x: a feature vector of dimension (1,3)
    '''
    return [1] + list(freq_df.loc[freq_df.vocab.isin(tokenized_tweet), ['pos', 'neg']].sum())

In [13]:
# Calling the function on test set
train_features = [extract_features(preprocess_tweet(tweet)) for tweet in x_train]
train_features[:5]

[[1, 2971, 3], [1, 565, 94], [1, 3910, 356], [1, 3849, 165], [1, 625, 5]]

In [14]:
# Training a Logistic Regression Model
lr = LogisticRegression()
lr.fit(train_features, y_train)

LogisticRegression()

In [15]:
# Test with a tweet
print(f'Tweet: {x_test[-1]}. \nPrediction: {lr.predict([extract_features(preprocess_tweet(x_test[-1]))])}')

Tweet: @eawoman As a Hull supporter I am expecting a misserable few weeks :-(. 
Prediction: [0.]


# Part 1: Logistic regression 


### Part 1.1: Sigmoid
We will use logistic regression for text classification. 
* The sigmoid function is defined as: 

$$ h(z) = \frac{1}{1+\exp^{-z}} \tag{1}$$

It maps the input 'z' to a value that ranges between 0 and 1, and so it can be treated as a probability. 

### Logistic regression: regression and a sigmoid

Logistic regression takes a regular linear regression, and applies a sigmoid to the output of the linear regression.

Regression:
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
Note that the $\theta$ values are "weights". 

Logistic regression
$$ h(z) = \frac{1}{1+\exp^{-z}}$$
$$z = \theta_0 x_0 + \theta_1 x_1 + \theta_2 x_2 + ... \theta_N x_N$$
We will refer to 'z' as the 'logits'.

### Part 1.2 Cost function and Gradient

The cost function used for logistic regression is the average of the log loss across all training examples:

$$J(\theta) = -\frac{1}{m} \sum_{i=1}^m y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)}))\tag{5} $$
* $m$ is the number of training examples
* $y^{(i)}$ is the actual label of the i-th training example.
* $h(z(\theta)^{(i)})$ is the model's prediction for the i-th training example.

The loss function for a single training example is
$$ Loss = -1 \times \left( y^{(i)}\log (h(z(\theta)^{(i)})) + (1-y^{(i)})\log (1-h(z(\theta)^{(i)})) \right)$$

* All the $h$ values are between 0 and 1, so the logs will be negative. That is the reason for the factor of -1 applied to the sum of the two loss terms.
* Note that when the model predicts 1 ($h(z(\theta)) = 1$) and the label $y$ is also 1, the loss for that training example is 0. 
* Similarly, when the model predicts 0 ($h(z(\theta)) = 0$) and the actual label is also 0, the loss for that training example is 0. 
* However, when the model prediction is close to 1 ($h(z(\theta)) = 0.9999$) and the label is 0, the second term of the log loss becomes a large negative number, which is then multiplied by the overall factor of -1 to convert it to a positive loss value. $-1 \times (1 - 0) \times log(1 - 0.9999) \approx 9.2$ The closer the model prediction gets to 1, the larger the loss.

* Likewise, if the model predicts close to 0 ($h(z) = 0.0001$) but the actual label is 1, the first term in the loss function becomes a large number: $-1 \times log(0.0001) \approx 9.2$.  The closer the prediction is to zero, the larger the loss.

#### Update the weights

To update the weight vector $\theta$, we will apply gradient descent to iteratively improve our model's predictions.  
The gradient of the cost function $J$ with respect to one of the weights $\theta_j$ is:

$$\nabla_{\theta_j}J(\theta) = \frac{1}{m} \sum_{i=1}^m(h^{(i)}-y^{(i)})x_j \tag{5}$$
* 'i' is the index across all 'm' training examples.
* 'j' is the index of the weight $\theta_j$, so $x_j$ is the feature associated with weight $\theta_j$

* To update the weight $\theta_j$, we adjust it by subtracting a fraction of the gradient determined by $\alpha$:
$$\theta_j = \theta_j - \alpha \times \nabla_{\theta_j}J(\theta) $$
* The learning rate $\alpha$ is a value that we choose to control how big a single update will be.


# Part 4: Test logistic regression

It is time to test the logistic regression function on some new input that the model has not seen before. 

#### Instructions: Write `predict_tweet`
Predict whether a tweet is positive or negative.

* Given a tweet, process it, then extract the features.
* Apply the model's learned weights on the features to get the logits.
* Apply the sigmoid to the logits to get the prediction (a value between 0 and 1).

$$y_{pred} = sigmoid(\mathbf{x} \cdot \theta)$$

In [16]:
lr.predict_proba([extract_features(preprocess_tweet(x_test[2]))])

array([[3.37774253e-12, 1.00000000e+00]])

In [17]:
def predict_tweet(tweet, model):
    '''
    Input: 
        tweet: a string
        model: trained model
    Output: the probability of a tweet being positive or negative
    '''
    return model.predict_proba([extract_features(preprocess_tweet(tweet))])

In [18]:
lr.coef_

array([[ 0.27177522,  0.00873689, -0.01001496]])

In [19]:
for tweet in ['I am happy', 'I am bad', 'this movie should have been great.', 'great', 'great great', 'great great great', 'great great great great']:
    print( f'{tweet} -> {predict_tweet(tweet, lr)[0]} -> {lr.predict([extract_features(preprocess_tweet(tweet))])}')

I am happy -> [0.12609435 0.87390565] -> [1.]
I am bad -> [0.46659585 0.53340415] -> [1.]
this movie should have been great. -> [0.1687119 0.8312881] -> [1.]
great -> [0.17057416 0.82942584] -> [1.]
great great -> [0.17057416 0.82942584] -> [1.]
great great great -> [0.17057416 0.82942584] -> [1.]
great great great great -> [0.17057416 0.82942584] -> [1.]


In [20]:
my_tweet = 'I am learning :)'
predict_tweet(my_tweet, lr)[0]

array([3.40039108e-12, 1.00000000e+00])

## Check performance using the test set
After training our model using the training set above, check how our model might perform on real, unseen data, by testing it against the test set.

In [21]:
def test_logistic_regression(test_x, test_y, model):
    """
    Input: 
        test_x: a list of tweets
        test_y: (m, 1) vector with the corresponding labels for the list of tweets
        model: Trained Logistic Regression Model
    Output: 
        accuracy: (# of tweets classified correctly) / (total # of tweets)
    """
    y_pred = model.predict([extract_features(preprocess_tweet(tweet)) for tweet in test_x])
    
    print('Confusion Matrix: \n', confusion_matrix(test_y, y_pred))
    print(classification_report(test_y, y_pred))
    
    return sum([x==y for x,y in zip(y_pred, test_y)]) / len(test_y)

In [22]:
# Print accuracy
test_logistic_regression(x_test, y_test, lr)

Confusion Matrix: 
 [[993   7]
 [  6 994]]
              precision    recall  f1-score   support

         0.0       0.99      0.99      0.99      1000
         1.0       0.99      0.99      0.99      1000

    accuracy                           0.99      2000
   macro avg       0.99      0.99      0.99      2000
weighted avg       0.99      0.99      0.99      2000



0.9935

Pretty good!

# Part 5: Error Analysis

In this part we will see some tweets that the model misclassified and to try to inspect why did the misclassifications happened and specifically what kind of tweets does the model misclassify?

In [23]:
# Some error analysis
print('Label Predicted Tweet')
for x,y in zip(x_test, y_test):
    y_hat = predict_tweet(x, lr)[0,1]

    if np.abs(y - (y_hat > 0.5)) > 0:
        print('THE TWEET IS:', x)
        print('THE PROCESSED TWEET IS:', preprocess_tweet(x))
        print('%d\t%0.8f\t%s' % (y, y_hat, ' '.join(preprocess_tweet(x)).encode('ascii', 'ignore')))

Label Predicted Tweet
THE TWEET IS: I'm playing Brain Dots : ) #BrainDots
http://t.co/cHl12JvuxN http://t.co/GGgU9PYEjI
THE PROCESSED TWEET IS: ["i'm", 'play', 'brain', 'dot', 'braindot', 't.co/chl12jvuxn', 't.co/gggu9pyeji']
1	0.36376919	b"i'm play brain dot braindot t.co/chl12jvuxn t.co/gggu9pyeji"
THE TWEET IS: I'm playing Brain Dots : ) #BrainDots
http://t.co/MifDDs7CQS http://t.co/WtIWoeATPj
THE PROCESSED TWEET IS: ["i'm", 'play', 'brain', 'dot', 'braindot', 't.co/mifdds7cq', 't.co/wtiwoeatpj']
1	0.36376919	b"i'm play brain dot braindot t.co/mifdds7cq t.co/wtiwoeatpj"
THE TWEET IS: I still fully intend to write as many game designs as possible while there. And an attack plan for the next 6 months. &gt;:D
THE PROCESSED TWEET IS: ['still', 'fulli', 'intend', 'write', 'mani', 'game', 'design', 'possibl', 'attack', 'plan', 'next', '6', 'month', '>:d']
1	0.47590703	b'still fulli intend write mani game design possibl attack plan next 6 month >:d'
THE TWEET IS: Remember that one time I d

# Part 6: Predict with new tweet

In [24]:
my_tweet = 'This is a ridiculously bright movie. The plot was terrible and I was sad until the ending!'
print(preprocess_tweet(my_tweet))
y_hat = predict_tweet(my_tweet, lr)[0,1]
print(y_hat)
if y_hat > 0.5:
    print('Positive sentiment')
else: 
    print('Negative sentiment')

['ridicul', 'bright', 'movi', 'plot', 'terribl', 'sad', 'end']
0.3171086659548087
Negative sentiment
